# Fundamental Trend Analysis
This notebook contains code for extracting fundamental data from an API and decomposing the series using the statsmodels library.

In [1]:
import pandas as pd
from datetime import date
from datetime import timedelta
import plotly.graph_objects as go

import requests
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

import numpy as np
from IPython.display import clear_output
import time
# Set plot space as inline for inline plots and qt for external plots
%matplotlib inline

In [2]:
# Obtain the API credentials from an external hidden file

from passwords import api_key

In [8]:
# Obtain today's date
today = date.today()

## 1. Obtain the Raw Data From the API

In [28]:
# A function to obtain fundamental data over time

#Utilizing "UniBit API" by Vincent Jiang
#Site: https://unibit.ai/product

#Errors may occur if credits run out, add an error code check to this function

def retrieve_ticker_fundamental_info(ticker, interval, start_date, end_date = False, statement_type = None):
    
    # Obtain today's date
    today = date.today()
    
    # A condition to set the end date to today if there is no end date given
    if not end_date:
        end_date = today
        
    if statement_type:
        statement_type = '&statement=' + statement_type
    
    #url = 'https://api.unibit.ai/v2/company/coreFinancials/?tickers={}&interval={}{}&startDate={}&endDate={}&dataType=json&accessKey={}'.format(ticker, interval, statement_type, start_date, end_date, api_key)
    url = 'https://api.unibit.ai/v2/company/coreFinancials/?tickers=LOW&interval=Q&statement=IS&startDate=2017-01-01&endDate=2020-09-12&dataType=json&accessKey={}'.format(api_key)
    response = requests.get(url)
    
    if response.status_code != 200:
        print("The response status code is: " + str(response.status_code))
        return None
    
    return response.json()

In [29]:
ticker = 'LOW'

# Interval can be Annual (A), Quarterly (Q), or Trailing 12 Month (TTM)
interval = 'Q'

# Statement type can be "IS" (Income Statement), "BS" (Balance Sheet), or "CF" (Cash Flow). Default is "IS"
statement_type = 'IS'

start_date = '2017-01-01'
end_date = today


fundamental_data = retrieve_ticker_fundamental_info(ticker, interval, start_date, end_date, statement_type)

In [30]:
def retrieve_fundamental_data_dataframe(ticker, fundamental_data):
    
    data_frame = pd.DataFrame(fundamental_data['result_data'][ticker])
    
    # Set the index to the 'date' column
    data_frame.set_index('report_date', inplace = True)
    
    data_frame.sort_index(inplace=True)
    
    return data_frame

In [31]:
retrieve_fundamental_data_dataframe(ticker, fundamental_data)

,operating_expense,interest_expense,selling_general_and_administration,gross_profit,income_before_tax,basic_eps,income_tax_expenses,other_income_expense_net,cost_of_revenue,operating_income_or_loss,net_income_continuous_operations,basic_average_shares,total_revenue,diluted_average_shares,diluted_eps,net_income,net_income_common_stockholders,research_and_development
report_date,,,,,,,,,,,,,,,,,,
2019-01-31,4605000192,161000000,4265999872,4036999936,-725000000,-1.03,99000000,-,11610000384,-568000000,-824000000,801000000,15646999552,801000000,-1.03,-824000000,-821000000,-
2019-04-30,4164000000,169000000,3862000128,5581000192,1255000064,1.31,209000000,-,12160000000,1416999936,1046000000,796000000,17741000704,797000000,1.31,1046000000,1043000000,-
2019-07-31,4359000064,181000000,4048000000,6739999744,2212000000,2.14,536000000,-,14252000256,2380999936,1676000000,781000000,20992000000,781000000,2.14,1676000000,1670000000,-
2019-10-31,4081999872,182000000,3772000000,5640000000,1380999936,1.36,332000000,-,11747999744,1558000000,1049000000,769000000,17387999232,770000000,1.36,1049000000,1046000000,-
2020-01-31,4023000064,186000000,3684999936,4981000192,775000000,0.67,265000000,-,11045999616,958000000,510000000,763000000,16026999808,764000000,0.66,510000000,508000000,-
2020-04-30,4521999872,210000000,4196000000,6512999936,1786000000,1.76,449000000,-,13162000384,1991000064,1336999936,755000000,19675000832,756000000,1.76,1336999936,1332999936,-
